In [2]:
import matplotlib.pyplot as plt # plotting library
import numpy as np # this module is useful to work with numerical arrays
import pandas as pd 
import random 
import torch
import torchvision
from torchvision import transforms
from torch.utils.data import Dataset,DataLoader,random_split
from torch.utils.data.sampler import SubsetRandomSampler
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
import math
import glob

In [3]:
files = glob.glob("/content/drive/MyDrive/AI_FYP/*1h.csv")

In [4]:
# train_data_full , train_ma_full= [],[]
# test_data_full , test_ma_full= [],[]

# train_data_full , train_ma_full= [],[]
# test_data_full , test_ma_full= [],[]
# for f in files:
#   data_df = pd.read_csv(f)

#   data_df = data_df[["close"]]

#   data = data_df.pct_change().dropna()

#   l = data.shape[0]
#   idx = int(l*0.8)
#   train_data= data[:idx]
#   test_data = data[idx:]

#   train_data_sli = np.lib.stride_tricks.sliding_window_view(train_data,(168+1,train_data.shape[1])).squeeze().astype(np.float32)

#   test_data_sli = np.lib.stride_tricks.sliding_window_view(test_data,(168+1,test_data.shape[1])).squeeze().astype(np.float32)

#   train_data_full.append(train_data_sli)

#   test_data_full.append(test_data_sli)

In [5]:
train_data_full , train_ma_full= [],[]
test_data_full , test_ma_full= [],[]
for f in files:
  data_df = pd.read_csv(f)

  data_df = data_df[["close"]].pct_change().dropna()
  ma_df = data_df


  for days in range(2,52):
    name = "SMA_"+str(days)
    ma_df[name]=ma_df["close"].rolling(days,min_periods=1).mean()
    data_df[name] = data_df["close"]


  ma = ma_df.drop(["close"],axis=1)
  data = data_df.drop(["close"],axis=1)

  ma = ma.values


  l = data.shape[0]
  idx = int(l*0.8)
  train_data,train_ma = data[:idx],ma[:idx]
  test_data,test_ma = data[idx:],ma[idx:]

  # train_data_sli = np.lib.stride_tricks.sliding_window_view(train_data,(168+1,train_data.shape[1])).squeeze().astype(np.float32)
  train_ma_sli = np.lib.stride_tricks.sliding_window_view(train_ma,(168+1,train_ma.shape[1])).squeeze().astype(np.float32)

  # test_data_sli = np.lib.stride_tricks.sliding_window_view(test_data,(168+1,test_data.shape[1])).squeeze().astype(np.float32)
  test_ma_sli = np.lib.stride_tricks.sliding_window_view(test_ma,(168+1,test_ma.shape[1])).squeeze().astype(np.float32)

  # train_data_full.append(train_data_sli)
  train_ma_full.append(train_ma_sli)

  # test_data_full.append(test_data_sli)
  test_ma_full.append(test_ma_sli)

In [6]:
train_ma_sli.shape

(10285, 169, 50)

In [7]:
train_data_full=np.concatenate(train_ma_full)
test_data_full=np.concatenate(test_ma_full)

In [8]:
train_x,train_y = train_data_full[:,:-1],train_data_full[:,-1]
test_x,test_y = test_data_full[:,:-1],test_data_full[:,-1]

train_y = train_y[:,0]
test_y = test_y[:,0]

In [9]:
class ClassDataset(Dataset):
    def __init__(self, input,output,threshold):
          self.input = torch.tensor(input)

          label = []
          p = 0
          n = 0
          z = 0
          for num in output:
            row = [0,0,0]
            # row = [0,0]
            if num>threshold:
              row[2]=1
              p+=1
            elif num<-threshold:
              row[0]=1
              n+=1
            else:
              row[1]=1
              z+=1
              pass
              # row[1]=1
            label.append(row)
          label = np.array(label).astype(np.float32)
          self.label = torch.tensor(label)
          self.output = torch.tensor(output)
          print("Total:",p+n+z)
          print("P:",p)
          print("Z:",z)
          print("N:",n)

    def __len__(self):
        return len(self.input)

    def __getitem__(self, idx):
        x = self.input[idx].T
        y = self.label[idx]

        return x,y

class Conv(nn.Module):
  def __init__(self, in_channels, out_channels, kernel_size, stride, padding):
    super().__init__()
    self.conv = nn.Conv1d(in_channels,out_channels,kernel_size=kernel_size,stride=stride,padding=padding)
    self.norm = nn.BatchNorm1d(out_channels)
    self.act = nn.LeakyReLU(True)
  
  def forward(self,x):
    y = self.conv(x)
    y = self.norm(y)
    y = self.act(y)
    return y

class line(nn.Module):
  def __init__(self,in_lay,out_lay):
    super().__init__()
    self.lin = nn.Linear(in_lay,out_lay)
    self.norm = nn.BatchNorm1d(out_lay)
    self.act = nn.LeakyReLU(True)
  
  def forward(self,x):
    y = self.lin(x)
    y = self.norm(y)
    y = self.act(y)
    return y

class Classifier(nn.Module):
    def __init__(self, **kwargs):
        super().__init__()
        self.model = nn.Sequential(
            Conv(50,32,7,1,0),
            Conv(32,8,7,1,0),
        )
        self.flat = nn.Flatten()

        self.linear = nn.Sequential(
            line(1248,640),
            line(640,320),
            line(320,64),
            line(64,3),
        )

    def forward(self,x):
      x = self.model(x)
      x = self.flat(x)
      x = self.linear(x)
      # print(x.shape)
      return x

In [10]:
train_set = ClassDataset(train_x,train_y,0.01)
test_set = ClassDataset(test_x,test_y,0.01)

BATCH_SIZE = 1024
train_loader = DataLoader(train_set,batch_size=BATCH_SIZE)
test_loader = DataLoader(test_set,batch_size=BATCH_SIZE)

Total: 51425
P: 6878
Z: 37922
N: 6625
Total: 12230
P: 1115
Z: 9818
N: 1297


In [11]:
for x,l in train_loader:
  print(x.shape)
  break

torch.Size([1024, 50, 168])


In [12]:
def multi_acc(y_pred, y_test):
    y_pred_softmax = torch.log_softmax(y_pred, dim = 1)
    _, y_pred_tags = torch.max(y_pred_softmax, dim = 1) 
    _, lab = torch.max(y_test,dim=1)   
    
    correct_pred = (y_pred_tags == lab).float()
    acc = correct_pred.sum() / len(correct_pred)
    
    acc = torch.round(acc * 100)
    
    return acc,y_pred_tags

In [13]:
epochs = 50
lr = 1e-6
use_cuda = 1
device = torch.device("cuda" if (torch.cuda.is_available() & use_cuda) else "cpu")

model = Classifier().to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=lr)

In [14]:
for epoch in range(10):  # loop over the dataset multiple times

    total_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()
        # print(inputs.size())
        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        total_loss += loss.item()
    total_loss /= len(train_loader.dataset)
    if epoch % 1 ==0:
      print('[{}/{}] Loss:'.format(epoch+1, epochs), total_loss)
      with torch.no_grad():
        model.eval()
        test_acc = 0
        for inputs,labels in test_loader:
          inputs = inputs.to(device)
          labels = labels.to(device)
          outputs = model(inputs)

          acc ,pred_tag= multi_acc(outputs, labels)
          test_acc += acc.item()
        test_acc = test_acc/len(test_loader)
      print('[{}/{}] Test Accuracy:'.format(epoch+1, epochs), test_acc)
    

print('[{}/{}] Loss:'.format(epoch+1, epochs), total_loss)

[1/50] Loss: 0.0013380905372979402
[1/50] Test Accuracy: 33.166666666666664
[2/50] Loss: 0.0010866607690124066
[2/50] Test Accuracy: 43.083333333333336
[3/50] Loss: 0.0010750047570591593
[3/50] Test Accuracy: 53.416666666666664
[4/50] Loss: 0.0010637203141445667
[4/50] Test Accuracy: 61.5
[5/50] Loss: 0.0010527858817583158
[5/50] Test Accuracy: 67.83333333333333
[6/50] Loss: 0.0010421822728262286
[6/50] Test Accuracy: 71.91666666666667
[7/50] Loss: 0.0010318921663602348
[7/50] Test Accuracy: 74.5
[8/50] Loss: 0.0010219005536963538
[8/50] Test Accuracy: 76.5
[9/50] Loss: 0.0010121937428624176
[9/50] Test Accuracy: 77.41666666666667
[10/50] Loss: 0.0010027599010180082
[10/50] Test Accuracy: 78.08333333333333
[10/50] Loss: 0.0010027599010180082


In [15]:
y_pred_softmax = torch.log_softmax(outputs, dim = 1)

In [16]:
torch.max(y_pred_softmax, dim = 1) 

torch.return_types.max(
values=tensor([-0.9236, -0.8728, -0.9332, -0.9826, -0.8802, -0.9901, -0.9270, -0.9294,
        -0.9268, -0.9434, -1.0064, -0.9421, -0.8933, -0.9959, -0.9302, -0.9392,
        -0.9343, -0.8915, -0.9185, -0.9613, -0.8725, -1.0149, -0.9571, -0.8924,
        -0.9687, -0.8904, -0.9156, -0.9690, -0.8622, -0.9811, -0.9197, -0.8352,
        -1.0110, -0.9777, -0.8813, -0.9111, -1.0301, -0.8637, -0.9305, -0.9431,
        -0.9487, -0.9464, -0.9787, -0.9466, -0.9917, -0.9378, -0.9404, -0.9577,
        -0.9557, -0.9550, -0.8801, -1.0020, -0.9475, -0.9111, -0.8598, -1.0152,
        -0.9056, -0.9559, -0.9515, -0.9625, -0.8736, -0.9549, -0.9304, -0.8740,
        -0.9100, -0.9894, -0.8694, -0.9643, -0.8793, -0.9915, -0.9565, -0.9703,
        -0.9007, -0.9093, -0.8982, -0.9391, -0.8681, -0.9342, -1.0000, -0.9651,
        -0.9146, -1.0422, -0.8701, -0.9694, -0.9115, -0.9389, -0.9422, -0.9251,
        -0.9474, -0.9763, -0.9725, -0.9397, -1.0027, -0.9562, -0.9370, -0.8939,
        -